In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------------
# Köppen Classification Function
# -------------------------------
def classify_koeppen(temp, precip, months):
    # Convert to numeric arrays
    temp = pd.to_numeric(temp, errors='coerce')
    precip = pd.to_numeric(precip, errors='coerce')

    # Basic climate metrics
    MAT = np.nanmean(temp)                # Mean Annual Temperature
    MAP = np.nansum(precip)               # Mean Annual Precipitation
    Tmin, Tmax = np.nanmin(temp), np.nanmax(temp)

    # Masks for seasons
    winter_mask = months.isin([10,11,12,1,2,3])
    summer_mask = months.isin(range(4,10))

    # Winter & Summer min/max precipitation
    PWmin = np.nanmin(precip[winter_mask])
    PWmax = np.nanmax(precip[winter_mask])
    PSmin = np.nanmin(precip[summer_mask])
    PSmax = np.nanmax(precip[summer_mask])

    Pmin = np.nanmin(precip)

    # Total precip by season
    MAP_summer = np.nansum(precip[summer_mask])
    MAP_winter = np.nansum(precip[winter_mask])

    # Aridity threshold
    if MAP_winter >= 0.7 * MAP:
        Pthreshold = 2 * MAT
    elif MAP_summer >= 0.7 * MAP:
        Pthreshold = 2 * MAT + 28
    else:
        Pthreshold = 2 * MAT + 14

    # -------------------------------
    # Köppen Climate Decision Tree
    # -------------------------------

    # Polar
    if Tmax < 10:
        return "ET" if Tmax > 0 else "EF"

    # Arid
    if MAP < 10 * Pthreshold:
        if MAP < 5 * Pthreshold:
            return "BWh" if MAT >= 18 else "BWk"
        return "BSh" if MAT >= 18 else "BSk"

    # Tropical (Tmin >= 18°C)
    if Tmin >= 18:
        if Pmin >= 60:
            return "Af"
        if Pmin >= 100 - MAP / 25:
            return "Am"
        if PSmin < 100 - MAP / 25:
            return "As"
        return "Aw"

    warm_months = np.sum(temp > 10)  # Count months > 10°C

    # Temperate (0 < Tmin < 18)
    if 0 < Tmin < 18:
        if PSmin < 40 and PSmin < PWmax / 3:
            return "Csa" if Tmax >= 22 else ("Csb" if warm_months >= 4 else "Csc")
        if PWmin < PSmax / 10:
            return "Cwa" if Tmax >= 22 else ("Cwb" if warm_months >= 4 else "Cwc")
        return "Cfa" if Tmax >= 22 else ("Cfb" if warm_months >= 4 else "Cfc")

    # Cold (Tmin <= 0)
    if Tmin <= 0:
        if PSmin < 40 and PSmin < PWmax / 3:
            if Tmin < -38: return "Dsd"
            return "Dsa" if Tmax >= 22 else ("Dsb" if warm_months >= 4 else "Dsc")
        if PWmin < PSmax / 10:
            if Tmin < -38: return "Dwd"
            return "Dwa" if Tmax >= 22 else ("Dwb" if warm_months >= 4 else "Dwc")
        if Tmin < -38: return "Dfd"
        return "Dfa" if Tmax >= 22 else ("Dfb" if warm_months >= 4 else "Dfc")

    return "Unknown"


# -------------------------------
# Load Data
# -------------------------------
file = Path(r"D:\climate change\monthly_averages.xlsx")

rain = pd.read_excel(file, sheet_name="Rainfall_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")
tmax = pd.read_excel(file, sheet_name="Tmax_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")
tmin = pd.read_excel(file, sheet_name="Tmin_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")

# Convert index to datetime
rain.index = pd.to_datetime(rain.index)
tmax.index = pd.to_datetime(tmax.index)
tmin.index = pd.to_datetime(tmin.index)

# Mean monthly temperature
temp = (tmax + tmin) / 2
months = pd.Series(temp.index.month)

# -------------------------------
# Run Classification per Station
# -------------------------------
results = []

for station in temp.columns:
    classification = classify_koeppen(
        temp[station].values,
        rain[station].values,
        months
    )
    results.append((station, classification))

# Save to file
df_out = pd.DataFrame(results, columns=["Station", "Koppen_Class"])
output = Path(r"D:\climate change\koeppen_classifications.csv")
df_out.to_csv(output, index=False)

print(f"✅ Köppen classifications saved to {output}")
# Köppen Climate Classification Script

✅ Köppen classifications saved to D:\climate change\koeppen_classifications.csv
